In [1]:
import h5py
import numpy as np
from numpy.random import default_rng
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

from smml.kernel import GaussianKernel

In [42]:
K = GaussianKernel()
X = np.random.rand(4,5)
X, X.shape

(array([[0.70862575, 0.66907621, 0.5672052 , 0.11716727, 0.83550667],
        [0.08118818, 0.18005281, 0.08683709, 0.80563281, 0.78716352],
        [0.51540697, 0.22313665, 0.41122192, 0.07790869, 0.5698869 ],
        [0.61933605, 0.25279966, 0.71515426, 0.86518066, 0.54043726]]),
 (4, 5))

In [63]:
X[:,np.newaxis,:].shape, X[np.newaxis,:,:]

((4, 1, 5),
 array([[[0.70862575, 0.66907621, 0.5672052 , 0.11716727, 0.83550667],
         [0.08118818, 0.18005281, 0.08683709, 0.80563281, 0.78716352],
         [0.51540697, 0.22313665, 0.41122192, 0.07790869, 0.5698869 ],
         [0.61933605, 0.25279966, 0.71515426, 0.86518066, 0.54043726]]]))

In [65]:
X[:,:,np.newaxis] - X

ValueError: operands could not be broadcast together with shapes (4,5,1) (4,5) 

In [55]:
np.diff(X, axis=0)

array([[-0.62743756, -0.4890234 , -0.4803681 ,  0.68846554, -0.04834315],
       [ 0.43421879,  0.04308384,  0.32438483, -0.72772412, -0.21727662],
       [ 0.10392908,  0.029663  ,  0.30393234,  0.78727197, -0.02944963]])

In [34]:
X_norm = np.linalg.norm(X) ** 2
X_norm

7.938685748177334

In [38]:
np.dot(X, X.T)

array([[1.98859028, 1.95392214, 1.31316319, 1.51380042],
       [1.95392214, 2.73804611, 1.66683538, 1.71664197],
       [1.31316319, 1.66683538, 1.57363879, 1.39981727],
       [1.51380042, 1.71664197, 1.39981727, 1.63841057]])

In [36]:
np.sum(X ** 2, axis=-1)

array([1.98859028, 2.73804611, 1.57363879, 1.63841057])

In [51]:
res1 = np.zeros((X.shape[0],X.shape[0]))

for i in range(0,X.shape[0]):
    for j in range(0,X.shape[0]):
        res1[i,j] = K(X[i], X[j])

res1

array([[1.        , 0.06857726, 0.51414861, 0.18277937],
       [0.06857726, 1.        , 0.17467321, 0.22130909],
       [0.51414861, 0.17467321, 1.        , 0.23470147],
       [0.18277937, 0.22130909, 0.23470147, 1.        ]])

In [50]:
X_norm = np.sum(X ** 2, axis=-1)
np.exp(-(X_norm[:,None] + X_norm[None,:] - 2 * np.dot(X, X.T)) / (2 * 0.25))

array([[1.        , 0.06857726, 0.51414861, 0.18277937],
       [0.06857726, 1.        , 0.17467321, 0.22130909],
       [0.51414861, 0.17467321, 1.        , 0.23470147],
       [0.18277937, 0.22130909, 0.23470147, 1.        ]])

In [90]:
PATH = '../../datasets/usps/usps.h5'
with h5py.File(PATH, 'r') as hf:
        train = hf.get('train')
        X_tr = train.get('data')[:]
        y_tr = train.get('target')[:]
        test = hf.get('test')
        X_te = test.get('data')[:]
        y_te = test.get('target')[:]

In [104]:
class Pegasos:
    def __init__(self, l=0.5, T=1000, K=GaussianKernel(), seed=42):
        self.l = l
        self.T = T
        self.K = K
        self.seed = seed

    def fit(self, X : np.ndarray, y : np.ndarray):
        if (self.l <= 0): 
            raise ValueError('Parameter lambda is not strictly positive')

        self.X_train = X
        self.y_train = y
        self.alphas = np.zeros(X.shape[0]) # Keep sparse data struct?
        rng = default_rng(self.seed)

        for t in tqdm(range(1, self.T + 1)):
            i = rng.integers(X.shape[0])
            s = np.sum(
                [self.alphas[j] * y[j] * self.K(X[j], X[i]) 
                 for j in range(X.shape[0])])
            if (y[i] / (self.l * t)) * s < 1:
                self.alphas[i] += 1

    def predict(self, X):
        #TODO: check if we can remove eta, sign should not change

        return np.array(
            [np.sign((1 / (self.l * self.T)) 
                     * np.sum([self.alphas[j] * self.y_train[j] * self.K(self.X_train[j], x) 
                               for j in range(self.X_train.shape[0])])) for x in X])

In [ ]:
class MulticlassPegasos:
    pass
    # should train multiple pegasos etc

In [91]:
y_tr

array([6, 5, 4, ..., 3, 0, 1])

In [97]:
y_tr_0 = np.where(y_tr == 0, 1, -1)

In [105]:
pegasos = Pegasos()

In [106]:
pegasos.fit(X_tr, y_tr_0)

100%|██████████| 1000/1000 [01:17<00:00, 12.86it/s]


In [107]:
y_pred = pegasos.predict(X_te)

In [108]:
y_pred

array([-1., -1., -1., ..., -1.,  1., -1.])

In [110]:
y_te_0 = np.where(y_te == 0, 1, -1)

In [111]:
accuracy_score(y_pred, y_te_0)

0.9825610363726955